In [ ]:
import warnings
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt
from random import randrange
sns.set(rc={'figure.figsize':(15,8)})

In [ ]:
df = pd.read_csv("./data/abalone.data", names=["sex", "length", "diameter", "height", "whole_weight", "shucked_weight", "viscera_weight", "shell_weight", "rings"])

In [ ]:
raw_abs = df.copy()
raw_abs

In [ ]:
raw_abs.loc[raw_abs['rings'].between(0, 7,inclusive='both'), 'ring_class'] = int(1)
raw_abs.loc[raw_abs['rings'].between(8, 10,inclusive='both'), 'ring_class'] = int(2)
raw_abs.loc[raw_abs['rings'].between(11, 15,inclusive='both'), 'ring_class'] = int(3)
raw_abs.loc[raw_abs['rings'] > 15, 'ring_class'] = int(4)
raw_abs['ring_class'] = raw_abs['ring_class'].astype(int)

In [ ]:
raw_abs.drop(columns = "rings", axis=1, inplace=True)


In [ ]:
raw_abs = raw_abs[(raw_abs['height']<0.4) & (raw_abs['height']>0.01)]

# Visualisations

### Distribtion of ring class

In [ ]:
raw_abs.head()

In [ ]:
raw_abs['ring_class'].value_counts().plot(kind='barh', figsize=(8,6))
plt.ylabel("Ring-class")
plt.xlabel("Class count")
plt.grid(False)
plt.title("Abalone ring-class count", y=1.02, fontsize = 18);

In [ ]:
raw_abs.hist()

In [ ]:
heatmap = sns.heatmap(raw_abs.corr(), annot=True, cbar=False, vmin=-1., vmax=1., cmap=sns.cm.rocket)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12)

# Building the model

In [ ]:
num_pipeline = Pipeline([
    ('Nomalisation', MinMaxScaler()),
    ])

sex_pipeline = Pipeline([
    ('ord_encoder', OrdinalEncoder(categories=[['M', 'F', 'I']]))
])

ringClass_pipeline = Pipeline([
    ('ringClass_1Hot', OneHotEncoder())
])

num_arribs = list(raw_abs.drop(columns=["sex", "ring_class"]))

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_arribs),
    ("sex", sex_pipeline, ['sex']),
    ("ringClass_1Hot", ringClass_pipeline, ['ring_class'])
])



In [ ]:
abs_prepared = pd.DataFrame(full_pipeline.fit_transform(raw_abs))
abs_prepared

In [ ]:
X = abs_prepared.iloc[:,:-4]
y = abs_prepared.iloc[:,-4:]

In [ ]:
# Single layer
def run_model(hidden_neurons, lr=0.01):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.4, random_state=42)

    model = keras.models.Sequential([
    keras.layers.Dense(hidden_neurons, input_shape = (8,), activation = "relu"),
    # keras.layers.Dense(32, activation = "relu"),
    keras.layers.Dense(4, activation = "softmax")
    ])

    model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.SGD(learning_rate=lr),
              metrics = ['accuracy']
              )

    history = model.fit(X_train, y_train, epochs=500, validation_split=0.1, verbose=0)

    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.title(" Single layer " + str(hidden_neurons) +" hidden nurons with " + str(round(history.history['accuracy'][-1], 4)*100) + " % Acc.")
    plt.savefig(".\\images\\1_layer_" + str(hidden_neurons) +"_hidden_nur.png", bbox_inches='tight')
    plt.show()

list(map(lambda x: run_model(hidden_neurons = x), [5,10,15,20]))

In [ ]:
model = keras.models.Sequential([
keras.layers.Dense(15, input_shape = (8,), activation = "relu"),
# keras.layers.Dense(32, activation = "relu"),
keras.layers.Dense(4, activation = "softmax")
])

model.compile(loss = 'categorical_crossentropy',
            optimizer = keras.optimizers.SGD(learning_rate=1e-1),
            metrics = ['accuracy']
            )

history = model.fit(X_train, y_train, epochs=500, validation_split=0.1, verbose=0)

In [ ]:
# Insert learning rate changes
# Single layer
def run_model(hidden_neurons, lr=0.01):
    model = keras.models.Sequential([
    keras.layers.Dense(hidden_neurons, input_shape = (8,), activation = "relu"),
    # keras.layers.Dense(32, activation = "relu"),
    keras.layers.Dense(4, activation = "softmax")
])

    model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.SGD(learning_rate=lr),
              metrics = ['accuracy']
              )

    history = model.fit(X_train, y_train, epochs=500, validation_split=0.1, verbose=1)

    

    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.title(" Single layer 15 hidden nurons and learning rate of " + str(lr) + " with " + str(round(history.history['accuracy'][-1], 4)*100) + " % Acc.")
    plt.savefig(".\\images\\1_layer_" + str(hidden_neurons) +"_hidden_nur.png", bbox_inches='tight')
    plt.show()

list(map(lambda x: run_model(hidden_neurons = 15, lr = x), [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]))

In [ ]:
# 2 layer model
def run_2_layer_model(hidden_neurons = 15, lr=1e-1):
    model = keras.models.Sequential([
    keras.layers.Dense(hidden_neurons, input_shape = (8,), activation = "relu"),
    keras.layers.Dense(hidden_neurons, activation = "relu"),
    keras.layers.Dense(4, activation = "softmax")])

    model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.SGD(learning_rate=lr),
              metrics = ['accuracy']
              )

    history = model.fit(X_train, y_train, epochs=500, validation_split=0.1, verbose=1)

    

    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.title("2 hidden layers " + str(hidden_neurons) +" hidden nurons each with " + str(round(history.history['accuracy'][-1], 2)*100) + " % Acc.")
    plt.savefig(".\\images\\1_layer_" + str(hidden_neurons) +"_hidden_nur.png", bbox_inches='tight')
    plt.show()
 
run_2_layer_model()

In [ ]:
# 2 layer model
def run_2_layer_model(hidden_neurons = 15, lr=1e-1):
    model = keras.models.Sequential([
    keras.layers.Dense(hidden_neurons, input_shape = (8,), activation = "relu"),
    keras.layers.Dense(hidden_neurons, activation = "relu"),
    keras.layers.Dense(hidden_neurons, activation = "relu"),
    keras.layers.Dense(4, activation = "softmax")])

    model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.SGD(learning_rate=lr),
              metrics = ['accuracy']
              )

    history = model.fit(X_train, y_train, epochs=500, validation_split=0.1, verbose=1)

    

    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.title("3 hidden layers " + str(hidden_neurons) +" hidden nurons each with " + str(round(history.history['accuracy'][-1], 2)*100) + " % Acc.")
    plt.savefig(".\\images\\1_layer_" + str(hidden_neurons) +"_hidden_nur.png", bbox_inches='tight')
    plt.show()
 
run_2_layer_model()

In [ ]:
# 3 layer model with ADAM
def run_2_layer_model(hidden_neurons = 15, lr=1e-1):
    model = keras.models.Sequential([
    keras.layers.Dense(hidden_neurons, input_shape = (8,), activation = "relu"),
    keras.layers.Dense(hidden_neurons, activation = "relu"),
    keras.layers.Dense(hidden_neurons, activation = "relu"),
    keras.layers.Dense(4, activation = "softmax")])

    model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.Adam(learning_rate=lr),
              metrics = ['accuracy']
              )

    history = model.fit(X_train, y_train, epochs=500, validation_split=0.1, verbose=1)

    

    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.title("3 hidden layers " + str(hidden_neurons) +" hidden nurons each with " + str(round(history.history['accuracy'][-1], 2)*100) + " % Acc.")
    plt.savefig(".\\images\\1_layer_" + str(hidden_neurons) +"_hidden_nur.png", bbox_inches='tight')
    plt.show()
 
list(map(lambda x: run_model(hidden_neurons = 15, lr = x), [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]))